In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
sumstat_dir = "/net/bmc-lab5/data/kellis/group/tanigawa/data/ukb21942/gwas_geno"

In [3]:
lym = f"{sumstat_dir}/blood_count/pop_10PCs/WB/INI30120.glm.tsv.gz"
ldl = f"{sumstat_dir}/blood_biochemistry/pop_10PCs/WB/INI20030780.glm.tsv.gz"
fev = f"{sumstat_dir}/spirometry/pop_10PCs/WB/INI1003063.glm.tsv.gz"
egfr = f"{sumstat_dir}/blood_biochemistry/pop_10PCs/WB/INI50030700.glm.tsv.gz"

In [8]:
dfs = []
for p in [lym, ldl, fev, egfr]:
    match = re.search(r'INI\d+', p)
    if match:
        trait_id = match.group()
        print(trait_id)
    else:
        raise ValueError(f"Could not extract trait ID from path: {p}")
    
    df = pd.read_csv(p, compression='gzip', sep='\t')
    
    if len(df[df['ALT'] == df['A1']]) != len(df):
        raise ValueError(f"Need to check for effect allele consistency!")
        
    df = df[['#CHROM', 'POS', 'A1', 'REF']].rename(columns={'#CHROM': 'CHR', 'POS': 'BP', 'REF': 'A2'})
    df['CHR'] = df['CHR'].apply(str) # convert all to string for consistency
    df = df[~df['CHR'].isin(['X', 'MT'])]  # remove 'X' and 'MT'
    df['CHR'] = df['CHR'].astype(int)
    dfs.append(df)
    #df.to_csv(f'{trait_id}.snps_to_finemap.txt.gz', sep='\t', compression='gzip', index=False)

INI30120


/tmp/ipykernel_120790/3349407695.py:10: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(p, compression='gzip', sep='\t')


INI20030780


/tmp/ipykernel_120790/3349407695.py:10: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(p, compression='gzip', sep='\t')


INI1003063


/tmp/ipykernel_120790/3349407695.py:10: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(p, compression='gzip', sep='\t')


INI50030700


/tmp/ipykernel_120790/3349407695.py:10: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(p, compression='gzip', sep='\t')


In [11]:
df_final = pd.concat(dfs).drop_duplicates()

In [12]:
#df_final.to_csv('snps_to_finemap.txt.gz', sep='\t', compression='gzip', index=False)

In [13]:
missing = pd.read_csv('priors/snps_with_var.gz.miss.gz', compression='gzip', sep='\t')

In [ ]:
## python /home/lucytian/data/0_SOFTWARE/polyfun/extract_snpvar.py --sumstats snps_to_finemap.txt.gz --out priors/snps_with_var.gz

In [17]:
df_final['ID'] = df_final[["CHR", "BP", "A1", "A2"]].astype(str).agg(":".join, axis=1)
missing['ID'] = missing[["CHR", "BP", "A_eff", "A2"]].astype(str).agg(":".join, axis=1)
df_final = df_final[~df_final['ID'].isin(missing['ID'])]
df_final.to_csv('snps_to_finemap.txt.gz', sep='\t', compression='gzip', index=False)